In [1]:
%load_ext autoreload
%autoreload 2

from util_0803_9 import *

import pandas as pd

## 실험 내용
1. 5주문 번들 가능 여부 테스트

### get_init 일반 함수를 이용한 5주문 번들 가능 여부 테스트

In [33]:
%load_ext autoreload
%autoreload 2

from util_0803_9 import *

problem_file = r'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_8.json'
# problem_file = r'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K50_1.json'

## ------------------- 기본 변수 할당 ---------------------------

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

inf = float('inf')

timelimit = 60

## ------------------- 초기 상태 할당 코드 -------------------------

car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

init_availables = [rider.available_number for rider in ALL_RIDERS]

all_bundles = []
for ord in ALL_ORDERS:
    new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
    car_rider.available_number -= 1
    all_bundles.append(new_bundle)

## ------------------  주문 조합 가능성 행렬 제작하기 -----------------------------

order_comb_possibility = [[True] * K for _ in range(K)]

for i in range(len(all_bundles)):
    for j in range(i + 1, len(all_bundles)):
        bundle1 = all_bundles[i]
        bundle2 = all_bundles[j]

        order_num1 = bundle1.shop_seq[0]
        order_num2 = bundle2.shop_seq[0]

        ip = try_merging_bundles_by_dist(K, DIST, ALL_ORDERS, ALL_RIDERS, bundle1, bundle2)

        if not ip:
            order_comb_possibility[order_num1][order_num2] = False
            order_comb_possibility[order_num2][order_num1] = False

optimized_order_perms = [dict(), dict(), dict()] # optimized_order_perms[rider_i] = {orders_sorted: 최적 번들}

min_init_cost = inf
min_init_cost_bundles = []
min_init_cost_rider_availables = []

weight_grid = [(1, -3, 0), (1, -1, 0), (0, -0.5, 0)]
temp_process_time = -1
for weight1, weight2, weight3 in weight_grid:
    bundles, result_rider_availables, cost = get_init_bundle(K, ALL_RIDERS, ALL_ORDERS, DIST, init_availables, weight1, weight2, weight3, try_merging_bundles_by_dist_possibles_only, order_comb_possibility, optimized_order_perms, False, 5)

    bundles, result_rider_availables = reassign_riders(K, ALL_ORDERS, ALL_RIDERS, DIST, init_availables, bundles)
    cost = sum((bundle.cost for bundle in bundles)) / K

    if cost < min_init_cost:
        min_init_cost = cost
        min_init_cost_bundles = bundles
        min_init_cost_rider_availables = result_rider_availables

    print(weight1, weight2, cost)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1 -3 3378.3519999999994
1 -1 3240.2880000000005
0 -0.5 3457.323


In [ ]:
1 -1 2850.1386666666663
1 -1.5 2807.6073333333334

In [34]:
min_init_cost_bundles

[Bundle(all_orders, BIKE, [1, 12, 0, 22], [12, 22, 0, 1], 60, True),
 Bundle(all_orders, BIKE, [3, 16, 11, 46], [11, 3, 16, 46], 77, True),
 Bundle(all_orders, CAR, [5, 9, 14, 43], [43, 5, 9, 14], 108, True),
 Bundle(all_orders, CAR, [6, 26, 35], [26, 35, 6], 110, True),
 Bundle(all_orders, BIKE, [49, 7, 39, 20], [7, 39, 20, 49], 84, True),
 Bundle(all_orders, BIKE, [8, 32, 54, 36, 47], [32, 36, 8, 54, 47], 99, True),
 Bundle(all_orders, CAR, [21, 10, 18, 44, 51], [10, 21, 18, 51, 44], 110, True),
 Bundle(all_orders, CAR, [58, 13, 72], [58, 13, 72], 165, True),
 Bundle(all_orders, BIKE, [59, 70, 79, 15], [59, 15, 70, 79], 81, True),
 Bundle(all_orders, CAR, [19], [19], 26, True),
 Bundle(all_orders, BIKE, [61, 23, 27, 66, 55], [23, 66, 27, 61, 55], 86, True),
 Bundle(all_orders, BIKE, [24, 62, 67, 80], [62, 80, 67, 24], 91, True),
 Bundle(all_orders, CAR, [25], [25], 26, True),
 Bundle(all_orders, BIKE, [52, 29, 30, 71], [71, 52, 30, 29], 99, True),
 Bundle(all_orders, CAR, [31, 63, 64